In [1]:
import time
from datetime import date
import pandas as pd
import os
import glob
import re
import CBE_utils as CBE

In [ ]:
input_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/input/"
output_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/output/"

annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"

input_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_input/"
output_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_output/"

annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"

In [3]:
def list_folders(directory):
    try:
        # List all entries in the specified directory
        entries = os.listdir(directory)
        
        # Filter out non-folder entries
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        return folders
    
    except FileNotFoundError:
        
        return f"The directory '{directory}' does not exist."
    
    except PermissionError:
        
        return f"Permission denied to access the directory '{directory}'."

In [10]:
def process_profile_metadata(profiles_dataframe, timestamp_dataframe, datetime_dataframe, source, batch_name):

    # Naming convention
    ## Metadata_plate_name: B1001
    ## Metadata_replicate_number: R1
    ## Metadata_Plate: B1001R1
    ## Metadata_plate_map_name: B1001_R1
    profiles_dataframe['Metadata_plate_name'] = profiles_dataframe['Metadata_Plate'].str[:-2]
    profiles_dataframe['Metadata_replicate_number'] = profiles_dataframe['Metadata_Plate'].str[-2:]
    profiles_dataframe['Metadata_plate_map_name'] = profiles_dataframe['Metadata_plate_name'] + "_" + profiles_dataframe['Metadata_replicate_number']
    
    profiles_dataframe = profiles_dataframe.rename(columns={"Metadata_Well": "Metadata_Well_randomized"})

    profiles_dataframe['Metadata_user'] = timestamp_dataframe['user'][0]
    
    # Gets the partner information from the source folder
    profiles_dataframe['Metadata_source'] = source
    
    # Gets the instrument information and imaging date and time
    profiles_dataframe['Metadata_instrument'] = timestamp_dataframe['instrument'][0]
    profiles_dataframe['Metadata_imaging_date'] = datetime_dataframe.date()
    profiles_dataframe['Metadata_imaging_time'] = datetime_dataframe.time()
    profiles_dataframe['Metadata_imaging_timezone'] = datetime_dataframe.strftime('%z')
    
    ## The Metadata_staining_date is the batch date
    # Staining date is define here
    batch_name_parsed = pd.to_datetime(batch_name, format='%y%m%d', errors='coerce')

    staining_date = 'NaN'

    # Check for invalid date
    if pd.isna(batch_name_parsed):
        staining_date = "invalid_date"
    else:
        staining_date = batch_name_parsed.strftime('%Y-%m-%d')

    profiles_dataframe['Metadata_staining_date'] = staining_date
    # profiles_dataframe['Metadata_staining_date'] = profiles_dataframe['Metadata_staining_date'].astype(str)

    return profiles_dataframe

In [11]:
def pad_numbers(value):
    match = re.match(r"([A-Z])(\d+)", value)
    if match:
        letter = match.group(1)
        number = match.group(2).zfill(2)  # Pad number to 2 digits
        return f"{letter}{number}"
    return value

In [12]:
source_list = list_folders(input_dir)

In [14]:
for source in source_list:
    
    print(source)
    
    source_path = os.path.join(input_dir, source)
    
    plate_list = list_folders(source_path)
    
    for plate_name in plate_list:
        
        print(plate_name)
    
        folder_path = os.path.join(source_path, plate_name)

        print(folder_path)

        if os.path.exists(folder_path):

            # The new data contains another layer with the batch date
            batch_list = list_folders(folder_path)

            for batch_name in batch_list:

                print(batch_name)

                batch_path = os.path.join(folder_path, batch_name)

                # IMPORTANT: there can be multiple merged files. 
                profiles_name = plate_name + "*_CP_Profiles_Aggregated.csv"
                profiles_file_list = glob.glob(os.path.join(batch_path, profiles_name))
        
                if len(profiles_file_list) > 0:
            
                    # read profiles file
                    try:
                
                        profiles = pd.read_csv(profiles_file_list[0])
                
                        # Check row count of original file
                        row_count = profiles.shape[0]
                        print(f"Info: {plate_name} has {row_count} rows")
                
                
                    except Exception as e:
                
                        print(f"Error reading {profiles_file_list[0]}: {str(e)}")
            
                    # read timestamp file
                    timestamp_file_path = os.path.join(batch_path, "TimeStamp.csv")
            
                    try:
                
                        timestamp = pd.read_csv(timestamp_file_path)
                    
                    except Exception as e:
                
                        print(f"Error reading {timestamp_file_path}: {str(e)}")
            
                    # get datatime info from the timestamp file
                    datetime = pd.to_datetime(timestamp['date'][0], errors='coerce')
                    processed_profiles = process_profile_metadata(profiles, timestamp, datetime, source, batch_name)

                    # load correct plate layout based on replication number
                    replicate_number = processed_profiles['Metadata_replicate_number'][0]

                    replicate_layout_name = None

                    if (replicate_number == 'R1'):
                
                        replicate_layout_name = 'Picklist_Replicate1.csv'
            
                    elif (replicate_number == 'R2'):
                
                        replicate_layout_name = 'Picklist_Replicate2.csv'
                
                    elif (replicate_number == 'R3'):
                
                        replicate_layout_name = 'Picklist_Replicate3.csv'
                
                    elif (replicate_number == 'R4'):
                
                        replicate_layout_name = 'Picklist_Replicate4.csv'
                
                    else: 
                
                        print(f"Error: {replicate_number} not valid")


                    print(f"{replicate_layout_name} selected")
                
                    plate_layout_path = os.path.join(annotation_dir, 'plate_layout', replicate_layout_name)
                
                    try:
                
                        plate_layout = pd.read_csv(plate_layout_path)
                
                        # Issue in the plate layout the well numbers are not padded
                        plate_layout['Destination Well'] = plate_layout['Destination Well'].apply(pad_numbers)
                        plate_layout['Source Well'] = plate_layout['Source Well'].apply(pad_numbers)
            
                    except Exception as e:
                
                        print(f"Error reading { plate_layout_path}: {str(e)}")

                    # rename the columns accordingly
                    processed_profiles['Metadata_picklist_name'] = replicate_layout_name

                    # Destination well are the randomized wells 
                    plate_layout = plate_layout.rename(columns={"Destination Well": "Metadata_Well_randomized"})

                    # Source well are the well IDs of the compound mother plate
                    plate_layout = plate_layout.rename(columns={"Source Well": "Metadata_Well"})
                    processed_profiles = pd.merge(processed_profiles, plate_layout, on='Metadata_Well_randomized')
            
                    # get info for pos. and neg. controls
                    controls_layout_path = os.path.join(annotation_dir, "pos_neg_ctrl.csv")
            
                    try:
                
                        controls_layout = pd.read_csv(controls_layout_path)
                
                        # Issue in the plate layout the well numbers are not padded
                        controls_layout['Metadata_Well'] = controls_layout['Metadata_Well'].apply(pad_numbers)
            
                    except Exception as e:
                
                        print(f"Error reading {controls_layout}: {str(e)}")
                
                    processed_profiles = pd.merge(processed_profiles, controls_layout, on='Metadata_Well')

                    # Bring metadata columns to the front
                    features = CBE.get_feature_vector(processed_profiles)
                    metadata_dataframe = set(processed_profiles.columns) - set(features)

                    processed_profiles = processed_profiles[list(metadata_dataframe) + list(features)]

                    # create directories
                    plate_map_name = processed_profiles['Metadata_plate_map_name'][0]
                    staining_date = processed_profiles['Metadata_staining_date'][0]
                    date_plate_map_name = staining_date + "_" + plate_map_name
                    
                    output_plate_path = os.path.join(output_dir, source, date_plate_map_name) 
                    os.makedirs(output_plate_path, exist_ok=True)
            
                    file_path = os.path.join(output_plate_path, plate_map_name)
                    filename = file_path + ".csv"

                    processed_profiles.to_csv(filename, index=False)

                    print(f"{staining_date} staining date")
                    print(f"{plate_map_name} loaded and resaved")
                    print("Finished plate")
            
                else: 
            
                    print(f"Error: {plate_name} no profiles")

USC
C1220R2
/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_input/USC/C1220R2
241003
Info: C1220R2 has 384 rows
Picklist_Replicate2.csv selected
2024-10-03 staining date
C1220_R2 loaded and resaved
Finished plate
USC02R1
/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_input/USC/USC02R1
240726
Info: USC02R1 has 384 rows
Picklist_Replicate1.csv selected
2024-07-26 staining date
USC02_R1 loaded and resaved
Finished plate
C1227R3
/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_input/USC/C1227R3
241016
Info: C1227R3 has 384 rows
Picklist_Replicate3.csv selected
2024-10-16 staining date
C1227_R3 loaded and resaved
Finished plate
USC01R1
/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_input/USC/USC01R1
240425
Info: USC01R1 has 384 rows
Picklist_Replicate1.csv selected
2024-04-25 staining date
USC01_R1 loaded and resaved
Finished pl